## Using the Planetary Computer's Data API

The Planetary Computer's [`/data` API](https://tomaugspurger.github.io/pc-data-api/lab/) provides an easy way to perform basic analytics and vizualize data hosted by the Planetary Computer, without having to deploy your own compute in Azure. This document gives an introduction to the data API.

## Background

One of the core principals of cloud-native geospatial is putting the compute next to the data. The Planetary Computer stores its data in Azure Blob Storage in the West Europe region, so that would mean using one of Azure [many compute services](https://docs.microsoft.com/en-us/azure/architecture/guide/technology-choices/compute-decision-tree) to set up your own compute in West Europe. That's  why we set up the [Planetary Computer Hub](https://planetarycomputer.microsoft.com/docs/overview/environment/): a *very* convenient way to get started with cloud-native geospatial from your own browser.

For some use-cases, however, logging into the Hub and starting a Python kernel isn't appropriate (displaying images on a webpage, for example). It's an essentially manual and interactive form of compute, and involves the (costly) process of starting a Jupyter server on a Virtual Machine in Azure. Even if there were a hot virtual machine or Azure Function ready and waiting, eliminating the startup cost, the hassel of deployment might not be worth it for the outcome (displaying an image, again).

That's why the Planetary Computer provides a `/data` API: to efficiently and conveniently serve these kinds of "simple" usecases. The `/data` API, along with our STAC API, is what powers our [Explorer](https://planetarycomputer.microsoft.com/docs/overview/explorer/).

The reference documentation for the data API is at https://planetarycomputer.microsoft.com/api/data/v1/docs.

One quick note about how this document is presented, which itself makes the point of why the `/data` API is useful: This is document is a [Jupyter](https://jupyter.org/) notebook, just a bit of speficially formatted JSON. It's being hosted at https://tomaugspurger.github.io/pc-data-api/lab?path=pc-data-api.ipynb, which served that notebook JSON along with a [jupyterlite](https://jupyterlite.readthedocs.io/en/latest/), a JupyterLab distribution that runs entirely built in the browser. This includes a Python kernel, backed by [Pyodide](https://pyodide.org/).

The key point is *there's no notebook server running in Azure next to the data.* Everything running in this notebook, including the Python code, is running local to wherever you are. We'll be making HTTP calls to workers sitting in Azure, next to the data. But the only things things sent back to you locally are (relatively) small images.

In [8]:
import piplite
import pyodide.http

await piplite.install(["pystac", "ipyleaflet"])
import pystac
import ipyleaflet
from IPython.display import JSON, GeoJSON, Image

## Display an Item

The simplest use of the `data` API looks similar to accessing a raw asset from Blob Storage. Many of our STAC items have a `rendered_preview` asset that's actually dynamically served by our `data` API.

In [4]:
r = await pyodide.http.pyfetch(
    "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343"
)
item = pystac.Item.from_dict(
    await r.json()
)
item.assets["rendered_preview"]

<Asset href=https://planetarycomputer-staging.microsoft.com/api/data/v1/item/preview.png?collection=sentinel-2-l2a&item=S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343&assets=visual&asset_bidx=visual%7C1%2C2%2C3&nodata=0>

A request to that URL will trigger a [TiTiler](https://devseed.com/titiler/) server to read raw data from Blob Storage, combine and transform it (according to the parameters in the URL, which you could customize) and return you the PNG for display.

In [5]:
Image(url=item.assets["rendered_preview"].href)

So we're able to display an asset using a client that only understands HTTP and JSON.

The `tilejson` asset is similar to `rendered_preview`, but is useful for putting the asset on a map.

In [10]:
item.assets["tilejson"]

<Asset href=https://planetarycomputer-staging.microsoft.com/api/data/v1/item/tilejson.json?collection=sentinel-2-l2a&item=S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343&assets=visual&asset_bidx=visual%7C1%2C2%2C3&nodata=0>

Making a request to that endpoint returns an object with a `tiles` url, which has everything filled in for this specific item.

In [11]:
r = await (await pyodide.http.pyfetch(item.assets["tilejson"].href)).json()
r

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://planetarycomputer-staging.microsoft.com/api/data/v1/item/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=sentinel-2-l2a&item=S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343&assets=visual&asset_bidx=visual%7C1%2C2%2C3&nodata=0'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [31.17569761, 8.95381176, 32.17948101, 9.95039568],
 'center': [31.677589310000002, 9.45210372, 0]}

In [12]:
tiles = r["tiles"][0]

That can be handed to any system that understands tilejson URLs, like `ipyleaflet` for example. Panning and zooming around the map will trigger requests to load new data.

In [13]:
center = (
    (item.bbox[1] + item.bbox[3]) / 2,
    (item.bbox[0] + item.bbox[2]) / 2
)

m = ipyleaflet.Map(
    center=center,
    controls=[ipyleaflet.FullScreenControl()],
    zoom=11,
)

m.add_layer(
    ipyleaflet.TileLayer(url=tiles)
)
m.scroll_wheel_zoom = True
m

Map(center=[9.45210372, 31.677589310000002], controls=(FullScreenControl(options=['position']), ZoomControl(op…

## Mosaics

Thus far, we've worked with just a single asset. The `/data` API also supports combining multiple assets into a single asset by [registering a STAC API search](https://planetarycomputer.microsoft.com/api/data/v1/docs#/PgSTAC%20Mosaic%20endpoints/register_search_mosaic_register_post). You define a provide a search defining the space, time, and other properties to include in the results and the `/data` API will combine the results.

We'll define the area of interest as a GeoJSON polygon. The "filter" is defined as [CQL-2 JSON](https://github.com/radiantearth/stac-api-spec/tree/master/fragments/filter).

In [14]:
# Define your area of interest
aoi = {
        "type": "Polygon",
        "coordinates": [
          [
            [
              29.036865234375,
              7.857940257224196
            ],
            [
              31.4813232421875,
              7.857940257224196
            ],
            [
              31.4813232421875,
              10.055402736564236
            ],
            [
              29.036865234375,
              10.055402736564236
            ],
            [
              29.036865234375,
              7.857940257224196
            ]
          ]
        ]
      }

filter_ = {
    "query":

filter_ = {
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
    {"op": "=", "args": [{"property": "collection"}, "sentinel-2-l2a"]},
    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 10]}
  ]
}
collection = "sentinel-2-l2a"

We can register this search with a `POST` request to the `/data/v1/mosaic/register` endpoint.

In [15]:
import json

body = json.dumps({"collection": collection, "filter": filter_})
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

In [16]:
r_register = await pyodide.http.pyfetch("https://planetarycomputer.microsoft.com/api/data/v1/mosaic/register", method="POST", body=body, headers=headers)
registered = await r_register.json()
registered

{'searchid': '125b5f36fbc3a2e80e011495798a9739',
 'links': [{'rel': 'metadata',
   'type': 'application/json',
   'href': 'https://planetarycomputer.microsoft.com/api/data/v1/mosaic/125b5f36fbc3a2e80e011495798a9739/info'},
  {'rel': 'tilejson',
   'type': 'application/json',
   'href': 'https://planetarycomputer.microsoft.com/api/data/v1/mosaic/125b5f36fbc3a2e80e011495798a9739/tilejson.json'}]}

That returns an object with a couple of links. We're interested in the `/tilejson.json` link, to visualize the results on a map.

In [17]:
tilejson_url = registered["links"][1]["href"]

We need a bit of extra information before we can visualize the results, though.

First, we need to include the STAC collection ID, which we already have.

Sencond, we need to tell the tiler how to convert the raw data to an image. Several libraries are involved here, including [TiTiler](http://devseed.com/titiler/), [rio-tiler](https://cogeotiff.github.io/rio-tiler/), and [rio-color](https://github.com/mapbox/rio-color). To keep things as simple as possible, we'll use another endpoint, `/mosaic/info` to get some good defaults that were set by the Planetary Computer team.

In [22]:
mosaic_info = await (
    await pyodide.http.pyfetch(f"https://planetarycomputer.microsoft.com/api/data/v1/mosaic/info?collection={item.collection_id}")
).json()
render_options = mosaic_info["renderOptions"][0]["options"]
render_options

'assets=B04&assets=B03&assets=B02&nodata=0&color_formula=Gamma RGB 3.2 Saturation 0.8 Sigmoidal RGB 25 0.35'

Finally, we can get our full tilejson URL.

In [26]:
render_options

'assets=B04&assets=B03&assets=B02&nodata=0&color_formula=Gamma RGB 3.2 Saturation 0.8 Sigmoidal RGB 25 0.35'

In [27]:
tiles = (
    await (
        await pyodide.http.pyfetch(f"{tilejson_url}?collection={item.collection_id}&{render_options}")
    ).json()
)["tiles"][0]
tiles

'https://planetarycomputer.microsoft.com/api/data/v1/mosaic/tiles/125b5f36fbc3a2e80e011495798a9739/WebMercatorQuad/{z}/{x}/{y}@1x?collection=sentinel-2-l2a&assets=B04&assets=B03&assets=B02&nodata=0&color_formula=Gamma+RGB+3.2+Saturation+0.8+Sigmoidal+RGB+25+0.35'

Which can be provided to `ipyleaflet`.

In [28]:
center = (8.956671496894216, 30.25909423828125)

m = ipyleaflet.Map(
    center=center,
    controls=[ipyleaflet.FullScreenControl()],
    zoom=9,
)

m.add_layer(
    ipyleaflet.TileLayer(url=tiles)
)
m.add_layer(
    ipyleaflet.GeoJSON(data=aoi, style={"fillOpacity": 0}),
)
m.scroll_wheel_zoom = True
m

Map(center=[8.956671496894216, 30.25909423828125], controls=(FullScreenControl(options=['position']), ZoomCont…

This is essentially how the [Planetary Computer Explorer](https://planetarycomputer.microsoft.com/explore) works. The `filter` is generated based on your browser's window and whatever filters you've toggled. Based on that user input, it generates the CQL2-json query, registers a search, builds a TileJSON request (using any visualization options you've set) and displays the result on the map.

## Next steps

This was a brief introduction to the `/data` API. For more, see the [reference documentation](https://planetarycomputer.microsoft.com/api/data/v1/docs). Feel free to share your creations using the `/data` API on the Planetary Computer [discussions board](https://github.com/microsoft/PlanetaryComputer/discussions/categories/show-and-tell).